##  MNIST For ML Beginners
이 튜토리얼은 머신러닝과 TensorFlow를 처음 사용하는 독자를위한 것입니다. MNIST가 무엇인지, softmax (다항 로지스틱) 회귀가 어떤 것인지 이미 알고 있다면, 조금 더 심화 된 튜토리얼을 선호 할 것입니다. 튜토리얼을 시작하기 전에 TensorFlow를 설치하십시오.

프로그램 방법을 배울 때 가장 처음 하는 일은 "Hello World"입니다. Hello World를 프로그래밍하는 것처럼 머신러닝에는 MNIST가 있습니다.

MNIST는 기초적인 컴퓨터 비전 데이터셋입니다. 이것은 아래와 같이 숫자 필기체 이미지로 구성되어 있습니다. 
![Turing's Device](https://www.tensorflow.org/images/MNIST.png)
또한 각 이미지마다 레벨링 되어 있어, 이 이미지가 어떤 숫자인지 말해 줍니다. 예를들어 위 이미지는 5, 0, 4, 1로 레벨링 되어 있습니다. 

이 튜토리얼에서는 이미지를 보고 어떤 수인지 예측하는 모델을 학습 할 것입니다. 우리 목표는 최첨단 성능을 구현하는 매우 정교한 모델을 학습하는 것이 아닙니다. 나중에 수행 할 코드를 제공 하겠지만, 오히려 TensorFlow 사용법에 발을 담그는 쪽에 가깝습니다. 따라서 우리는 Softmax Regression이라 불리는 매우 간단한 모델로 시작할 것입니다.

이 튜토리얼의 실제 코드는 매우 짧으며, 모든 흥미로운 내용은 단 3 줄에 있습니다. 그러나 TensorFlow가 작동하는 방법과 핵심 머신러닝 개념 모두에 대한 아이디어를 이해하는 것이 매우 중요합니다. 이 때문에 우리는 코드를 통해 차근차근 접근 할 것입니다.

###  About this tutorial
이 튜토리얼에서는 ** mnist_softmax.py ** 코드를 라인별로 설명하겠습니다. 

이 튜토리얼는 다음과 같은 몇 가지 방법으로 사용할 수 있습니다. 
* 각 행의 설명을 읽으면서 한 줄씩 각 코드를 복사하여 Python 환경에 붙여 넣으십시오. 
* 설명을 읽기 전이나 후에 mnist_softmax.py 파이썬 파일 전체를 실행하고, 이 튜토리얼을 사용하여 명확하지 않은 코드 행을 이해하십시오.

이 튜토리얼에서 우리가 달성 할 수있는 것 : 
* MNIST 데이터 및 softmax 회귀에 대한 학습. 
* 이미지의 모든 픽셀을 보면서 숫자를 인식하는 모델을 생성. 
* TensorFlow를 사용하여 수천 가지 데이터를 "훑어보고"숫자를 인식하도록 모델을 학습
* 테스트 데이터로 모델의 정확성을 확인.

###  The MNIST Data

MNIST 데이터는 [Yann LeCun의 웹 사이트](http://yann.lecun.com/exdb/mnist/)에서 제공됩니다. 이 튜토리얼의 코드를 복사하여 붙여 넣는 경우, 다음 두 줄의 코드를 사용하여 데이터를 자동으로 다운로드하고 읽습니다.

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


MNIST 데이터는 학습 데이터(mnist.train)의 55,000개, 테스트 데이터 (mnist.test)의 10,000개 및 유효성 검사 데이터 (mnist.validation) 5,000개와 같이 세 부분으로 나뉩니다. 이러한 분할은 매우 중요합니다. 머신러닝에서 필수적입니다. 우리가 학습 시키지 않은 별도의 데이터를 통해 학습 내용이 실제로 일반화되었는지 확인할 수 있습니다.

앞에서 언급했듯이, 모든 각 MNIST 데이터는 두 부분으로되어 있습니다 : 손으로 쓴 자릿수의 이미지와 해당 레이블. 이를 이미지 "x"와 레이블 "y"라고 부를 것입니다. 트레이닝 세트와 테스트 세트 모두 이미지와 해당 레이블을 포함합니다. 예를 들어 트레이닝 데이터의 이미지는 mnist.train.images이고 트레이닝 데이터의 라벨은 mnist.train.labels입니다.

각 이미지는 28x28 픽셀이며, 이는 2차원 배열로 표현됩니다. 
<img src="https://www.tensorflow.org/images/MNIST-Matrix.png" width="500">
이 배열을 28x28 = 784 숫자의 벡터로 전개 할 수 있습니다. 우리가 이미지들 간에 일관성 있게 한 배열을 전개하게하는 방법은 중요하지 않습니다. 이러한 관점에서 볼 때, MNIST 이미지는 784 차원의 벡터 공간에서 매우 조밀한 구조(경고 : 계산 집약적 시각화)로 표현된 점들에 불과합니다.

데이터를 전개하면 이미지의 2D 구조에 대한 정보가 삭제됩니다. 잘못된 방법일 수도 있습니다. 최고의 컴퓨터 비전 방법은 이차원 구조를 그대로 이용하며, 다음 튜토리얼에서 다룰 것 입니다. 그러나 우리가 여기서 사용할 간단한 방법, softmax 회귀(아래 정의)는 그렇지 않습니다.

그 결과 ** mnist.train.images **는 [55000, 784] 모양의 텐서 (n 차원 배열)로 표협됩니다. 첫 번째 차원은 이미지 목록에 대한 인덱스이고 두 번째 차원은 각 이미지의 각 픽셀에 대한 인덱스입니다. 텐서의 각 엔트리는 특정 이미지의 특정 픽셀에 대해 0과 1 사이의 픽셀 값입니다.
<img src="https://www.tensorflow.org/images/mnist-train-xs.png" width="500">
각 이미지는 그에 대응하는 0~9 사이의 숫자 레이블을 가지고 있습니다. 

이 튜토리얼의 목적을 위해, 우리는 레이블을 "원 핫 벡터 (one-hot vectors)"로 표현합니다. 원 핫 벡터는 대부분 값이 0이고 한 인덱스에서 1 인 벡터입니다. 이 경우 n 번째 숫자는 $n$ 번째 차원에서 1 인 벡터로 표시됩니다. 예를 들어, 3은 $[0,0,0,1,0,0,0,0,0,0]$입니다. 따라서 ** mnist.train.labels **는 $[55000, 10]$ 형태의 배열입니다.

<img src="https://www.tensorflow.org/images/mnist-train-ys.png" width="500">
이제 실제 모델을 만들 준비가 끝났습니다. 



###  Softmax Regressions
우리는 MNIST의 모든 이미지가 0에서 9 사이의 필기테 숫자임을 알고 있습니다. 따라서 주어진 이미지가 될 수 있는 것은 10 가지 밖에 없습니다. 우리는 이미지를보고 각 숫자가 될 확률을 얻길 원합니다. 예를 들어, 우리 모델은 9의 그림을보고 80 %가 9라고 확신 할 수 있지만, 8일 확률도 5%이므로(윗 부분의 둥근모양 때문에) 다른 모든 경우에도 약간의 확률이 있습니다. 100% 확실한 경우는 없습니다. 

이것은 softmax 회귀의 자연스럽고 단순한 모델에 대한 전형적인 사례입니다. 여러 다른 것들 중 하나인 객체에 확률을 할당하는 경우, softmax는 0과 1 사이의 값으로 구성된 리스트를 제공하기 때문에 위과 같은 사례에 softmax를 사용하는 것은 매우 적합합니다. 더 정교한 모델을 다루는 다음 과정에서도 마지막 단계는 softmax의 레이어가 될 것입니다.

softmax 회귀는 두 부분으로 구성되어 있습니다. 먼저 특정 클래스에서 입력에 대한 출력 값을 더한 다음, 그 값을 확률로 변환합니다. 

주어진 이미지가 특정 클래스에 있다는 속하는 정도를 계산하기 위해 픽셀 값의 가중치을 계산합니다. 높은 값를 가진 픽셀이 해당 클래스에있는 이미지에 대한 증거(evidence)라면 가중치는 음수이고, 그 반대라면 양수로 표현됩니다. 다음 다이어그램은 이러한 각 클래스에 대해 학습 한 모델의 가중치를 보여줍니다. 빨간색은 음수를 나타내고 파란색은 양수를 나타냅니다.
<img src="https://www.tensorflow.org/images/softmax-weights.png" width="500">
우리는 바이어스(bias)이라고 불리는 추가적인 증거를 추가합니다. 기본적으로 우리는 어떤 것들은 입력과 독립적수 있다고 말할 수 있습니다. 입력 $x$의 주어진 클래스 $i$에 대한 증거는 다음과 같이 표현됩니다. 
$$\text{evidence}_i = \sum_j W_{i,~ j} x_j + b_i$$
여기서 $W_i$와 $b_i$는 클래스 $i$의 가중치와 바이어스며, $j$는 입력 이미지 $x$ 내 픽셀의 인덱스입니다. "softmax"함수를 통해 evidence를 확률 $y$로 바꿉니다.   
$$y = \text{softmax}(\text{evidence})$$
여기서 softmax는 우리의 선형 함수의 출력을 원하는 형태, 이 경우에는 10 가지 이상의 확률 분포로 형성하는 "활성화"또는 "링크"함수로 사용됩니다. 값들을 집계하여 각 클래스 할당 될 확률로 변형하는 것으로 생각할 수 있습니다. 이는 다음과 같이 정의됩니다.
$$\text{softmax}(x) = \text{normalize}(\exp(x))$$
식을 풀어쓰면 다음과 같습니다. 
$$\text{softmax}(x)_i = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

그러나 softmax를 입력을 지수화 한 다음 정규화하는 첫 번째 식으로 생각하는 것이 더 도움이 됩니다. 지수화는 하나의 증거값이 모든 주어진 가중치를 지수적으로 증가 시킨다는 것을 의미합니다. 반대로 하나의 증거값도 없다는 것은 초기 가중치 중 일부를 얻는다는 것을 의미합니다. 그어떠한 경우에도 0 또는 음수의 가중치를 갖지 않습니다. Softmax는 이러한 가중치를 합이 1이 되도록 정규화하여 유효한 확률 분포를 형성합니다. 

다음과 같이 $x$가 훨씬 더 많은 경우의 softmax 회귀 분석도 볼 수 있습니다. 각 출력에 대해 x의 가중치 합계를 계산하고 바이어스를 추가 한 다음 softmax를 적용합니다.
<img src="https://www.tensorflow.org/images/softmax-regression-scalargraph.png" width="500">
식으로 표현하면 다음과 같습니다. 
<img src="https://www.tensorflow.org/images/softmax-regression-scalarequation.png" width="500">
백터화 과정을 통해 행렬 곱 및 백터 합으로 표현할 수도 있습니다. 이러한 표현법은 계산을 효율적으로 하는데 도움을 줍니다. 
<img src="https://www.tensorflow.org/images/softmax-regression-vectorequation.png" width="500">
더 간단히 그냥 아래와 같이 쓸 수도 있습니다. 
$$y = \text{softmax}(Wx + b)$$
이제 이를 Tensorflow에서 사용하게 바꿔 보겠습니다. 





### Implementing the Regression
파이썬에서 효율적으로 행렬 곱셈과 같은 값 비싼 수치 연산을 하기위해, 일반적으로 다른 언어로 구현 된 매우 효율적인 코드로 구현된 NumPy와 같은 라이브러리를 사용합니다. 불행하게도, 모든 작업을 파이썬으로 다시 전환하면 많은 오버 헤드가 발생할 수 있습니다. GPU 또는 분산 방식으로 계산을 실행하려면 데이터를 전송하는 데 비용이 많이 드는 경우가 특히 많습니다.

TensorFlow는 파이썬 밖에서도 복잡한 작업을 수행하지만 이러한 오버 헤드를 피하기 위해 한 걸음 더 나아갑니다. TensorFlow는 Python과 독립적으로 값 비싼 단일 작업을 실행하는 대신, Python 외부에서 상호 작용하는 그래프의 연산을 수행합니다. (이와 같은 접근법은 몇몇 다른 머신러닝 라이브러리에서도 볼 수 있습니다.)

텐서플로우를 사용하기 위해, 먼저 이를 임포트합니다. 


In [3]:
import tensorflow as tf

우리는 심볼릭(symbolic) 변수를 조작하여 이러한 상호 작용 연산을 표현합니다. 한번 만들어 보겠습니다.

In [4]:
x = tf.placeholder(tf.float32, [None, 784])

**x** 는 특정 변수가 아닙니다. 이것은**placeholder**이며 Tensorflow에 Run을 수행 할 때 특정 값이 입력됩니다. 우리는 무수히 많은 MNIST 이미지를 입력으로 사용하기를 원합니다. 각각의 이미지는 784 차원의 벡터로 전개됩니다. 이것을 **[None, 784]** 형태의 2 차원 텐서 froting-point로 나타냅니다. 여기서 None은 차원의 크기가 임의 일 수 있음을 의미합니다.

우리 모델의 가중치와 바이어스도 필요합니다. 이러한 요소를 위와 같은 추가 입력으로 처리한다고 생각 할 수도 있지만 TensorFlow는 이를 처리하는 더 좋은 방법을 제공합니다. **Variable** 은 TensorFlow의 상호 작용 그래프 내 수정 가능한 텐서입니다. 그것은 계산에 사용되거나 심지어 중간에 수정 될 수도 있습니다. 머신러닝 어플리케이션에서 일반적으로 모델 파라미터는 변수입니다.

In [6]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

** tf.Variable **의 초기 값을 주어 이러한 변수(Variable)를 만듭니다.이 경우 W와 b 모두를 0으로 채워진 텐서로 초기화합니다. 우리는 W와 b값을 학습해야하기 때문에 초기 값이 무엇인지는 중요하지 않습니다.

W를 784 차원의 이미지 벡터에 곱해서, 여러 클래스를 나타내는 10차원 클래스 벡터가 생성해야 되므로 W는 [784, 10]의 모양을가집니다. b는 [10] 형태를 가지므로 출력에 추가 할 수 있습니다. 이제 모델을 구현할 수 있습니다. 

그것을 정의하는데는 단 한 줄이면 됩니다. 

In [7]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

먼저, ** tf.matmul (x, W) **식으로 x에 W를 곱합니다. 이것은 $Wx$로 표현되는 곱과 반대 순서로 표협된니다. x가 여러 입력을 갖는 2D 텐서이므로 이를 처리하기 위한 작은 트릭입니다. 그런 다음 b를 더하고 마지막으로 ** tf.nn.softmax **를 적용합니다.

이게 전부입니다. 몇 줄의 설정을 위한 코드 다음, 모델을 정의하는 데는 한 줄만 사용했습니다. 이는 TensorFlow가 softmax 회귀를 쉽게 만들 수 있도록 설계 되었기 때문이 아닙니다. 머신러닝 모델에서 물리 시뮬레이션에 이르기까지 다양한 종류의 수치 계산을 설명하는 매우 다양한 방법 중 하나 일 뿐입니다. 정의 된 모델은 컴퓨터의 CPU, GPU 및 스마트폰과 같은 다양한 장치에서 실행할 수 있습니다!

###  Training
모델을 훈련시키기 위해서는 모델이 잘 동작한다는 것을 정의해야합니다. 실제 머신러닝에서 우리는 보편적으로 모델이 얼마나 안 좋게 동작하는지를 정의합니다. 우리는 이를 비용(Cost) 또는 손실(Loss)라고 부르며, 이는 디자인 한 모델이 우리가 원하는 결과와 얼마나 차이나는지 나타냅니다. 우리는 오류를 최소화하려고 노력하며, 오류 마진이 작을수록 모델이 더 좋습니다.

모델의 손실을 결정하는 매우 일반적인 함수 중 하나는 "크로스 엔트로피 (cross-entropy)"입니다. 크로스 엔트로피 개념은 정보 이론에서 정보를 압축하는데서 시작되었지만, 도박에서 기계 학습에 이르기까지 많은 분야에서 사용됩니다. 그것은 다음과 같이 정의됩니다.
$$H_{y'}(y) = -\sum_i y'_i \log(y_i)$$
여기서 $y$는 우리의 예측 확률 분포고, $y'$는 참(정답) 분포 (숫자 레이블이있는 원 핫 벡터)입니다. 크로스 엔트로피는 우리의 예측이 실제 결과를 설명하는 데 얼마나 비효율적인지를 측정합니다. 크로스 엔트로피에 대한 자세한 내용은 이 튜토리얼의 범위를 벗어나지만 이해할 가치가 있습니다. 

크로스 엔트로피를 구현하려면, 정답을 입력하기위해 새로운 placeholder를 추가해야합니다.

In [8]:
y_ = tf.placeholder(tf.float32, [None, 10])

다음과 같이 크로스 앤트로피 함수를 구현 할 수 있습니다. $-\sum y'\log(y)$

In [9]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

먼저, **tf.log**는 y의 각 원소의 로그 값을 계산합니다. 다음으로, y_의 각 요소에 tf.log (y)의 해당 요소를 곱합니다. 그런 다음 tf.reduce_sum은 reduction_indices = [1] 파라미터로 인해 y의 두 번째 차원의 요소를 더합니다. 마지막으로 **tf.reduce_mean**은 배치의 모든 데이터에 대한 평균을 계산합니다.

소스 코드에서는 수치 적으로 불안정하기 때문에 위 공식을 사용하지 않습니다. 대신, softmax activation을 계산하는 수치적으로 더 안정된 내부 함수 ** tf.nn.softmax_cross_entropy_with_logits**에 비 표준화 된 로그를 적용합니다 (예 : tf.matmul (x, W) + b)에 **softmax_cross_entropy_with_logits **를 호출합니다). 여러분 코드에서는 tf.nn.softmax_cross_entropy_with_logits을 대신 사용하기바랍니다. 

이제 우리 모델이 하고자 하는 것이 무엇인지 알기 때문에, TensorFlow가 그렇게하도록 훈련시키는 것은 매우 쉽습니다. TensorFlow는 전체 Compute Graph를 알고 있기 때문에, 역전파(backpropagation) 알고리즘을 자동으로 사용하여 손실이 최소화 되도록 변수에 효과적인 영향을 줄 수 있습니다. 그런 다음 최적화 알고리즘을 선택하여 변수를 조정하고 손실을 줄일 수 있습니다.

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

이 경우 TensorFlow에 learning rate 0.5인 gradient descent 알고리즘을 사용하여 ** ross_entropy**를 최소화하도록 요청합니다. 그라디언트 디센트는 TensorFlow는 각 변수를 비용을 줄이는 방향으로 조금씩 이동 시키는 간단한 방법입니다. 그러나 TensorFlow는 다른 많은 최적화 알고리즘도 제공합니다. 한 줄로 간단하게 그것 들 중 하나를 사용 할 수 있습니다.

여기서 TensorFlow가 실제로 수행하는 작업은 역전파 및 그라디언트 디센트를 구현하는 새 연산을 그래프에 추가하는 것입니다. 그런 다음 실행하면 연산을 다시 실행하여 그라데이션 디센트 학습 단계를 수행하고 변수를 약간 조정하여 손실을 줄입니다.

**InteractiveSession**을 통해 모델에 대한 학습을 시작(launch)할 수 있습니다. :

In [11]:
sess = tf.InteractiveSession()

먼저 생성된 변수들을 초기화 해주는 연산을  만들어야 합니다.  

In [12]:
tf.global_variables_initializer().run()

이제 학습을 시작합니다. 학습을 1000회 실행하겠습니다. 

In [13]:
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

루프의 각 단계마다 우리의 트레이닝 세트에서 100 개의 무작위 데이터 포인트에 대한 "배치(batch)"를 얻습니다. **place_holder**를 대체하기 위해 배치 데이터에서 **train_step** 할당을 실행합니다. 

무작위 데이터의 작은 배치를 사용하여 확률적 학습을 진행합니다. 이러한 방법을 확률적(stochastic) 그라디언트 디센트라고합니다. 이상적으로, 우리는 우리가 해야 할 일에 대해 더 잘 이해할 수 있기 때문에 모든 학습 단계에 전체 데이터를 사용하고자하지만, 이러한 방법은 비용이 많이 듭니다. 대신, 우리는 매번 다른 부분 집합을 사용합니다. 이렇게 하는 것은 연산에 대한 비용이 적게들며 많은 이점을 가지고 있습니다.

### Evaluating Our Model

우리 모델은 얼마나 잘 동작할까요?

음, 우선 우리가 올바른 라벨을 예측했는지 알아 보겠습니다. **tf.argmax**는 어떤 축을 따라 텐서에서 가장 높은 엔트리의 인덱스를 제공하는 매우 유용한 함수입니다. 예를 들어, **tf.argmax (y, 1)**는 우리 모델이 각 입력에 대해 가장 가능성이 있다고 생각하는 레이블이고, **tf.argmax(y_, 1)**는 정답 레이블입니다. **tf.equal**을 사용하여 예측이 정답과 일치하는지 확인할 수 있습니다.

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

그 결과로 우리에게 이진 리스트를 줍니다. 어떤 부분이 올바른지 결정하기 위해 floting-point로 캐스팅 한 다음 평균을 취합니다. 예를 들어 $[True, False, True, True]$는 $[1,0,1,1]$이 되어, 평균 $0.75$가됩니다.

In [15]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

끝으로 테스트 데이터에 대한 정확도를 측정하겠습니다. 

In [16]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9169


약 92%가 나올 것입니다. .

이 정도면 괜찮지 않나요? 사실 이것은 별로 좋지 않은 수준입니다. 우리가 매우 간단한 모델을 사용했기 때문입니다. 약간만 바꾸면 97%의 정확도를 얻을 수 있습니다. 가장 좋은 모델의 정확도는 99.7% 이상이나 됩니다. 여기에 관한 더 많은 정보는 [list of results](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)에서 확인 할 수 있습니다. 

중요한 것은 우리가 이 모델로부터 머신러닝에 대한 기초를 배웠다는 것 입니다. 그래도 이 결과에 대해 조금 안타까움이 느껴진다면, 다음 튜토리얼에서 TensorFlow를 사용하여 보다 정교한 모델을 작성하는 방법을 살펴보십시오!